#### 1. 划分数据集

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split  
import pandas as pd
#分块处理大文件
# chunk_iter = pd.read_csv("data/train_data.csv", chunksize=10000)
# chunks = [chunk for chunk in chunk_iter]
# train_data = pd.concat(chunks)
train_data = pd.read_csv("data/processed_train.csv")
test_data = pd.read_csv("data/processed_test.csv")
train_data = train_data.drop(columns='issueDate')
test_data = test_data.drop(columns='issueDate')
'''#截取前10000行数据
train_data=train_data.iloc[:10000,2:] '''
'''
# 划分特征变量与目标变量
X=train_data.drop(columns='isDefault')
Y=train_data['isDefault']
# 划分训练及测试集
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=0)
'''
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 22 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   loanAmnt            800000 non-null  int64  
 1   term                800000 non-null  int64  
 2   interestRate        800000 non-null  int64  
 3   installment         800000 non-null  int64  
 4   grade               800000 non-null  int64  
 5   subGrade            800000 non-null  int64  
 6   employmentTitle     800000 non-null  float64
 7   homeOwnership       800000 non-null  int64  
 8   annualIncome        800000 non-null  int64  
 9   verificationStatus  800000 non-null  int64  
 10  isDefault           800000 non-null  int64  
 11  dti                 800000 non-null  int64  
 12  ficoRangeLow        800000 non-null  float64
 13  ficoRangeHigh       800000 non-null  float64
 14  revolUtil           800000 non-null  int64  
 15  earliesCreditLine   800000 non-nul

In [4]:
from sklearn.model_selection import train_test_split
 
feature_list = [col for col in train_data.columns if col != "isDefault"]
for fea in feature_list:
    train_data[fea] = train_data[fea].astype('int64')
    test_data[fea] = test_data[fea].astype('int64')
X_train, X_validation, y_train, y_validation = train_test_split(train_data.loc[:, feature_list], train_data.loc[:, 'isDefault'], test_size=0.2 , random_state=2000)

#### 2.权重统计

In [ ]:
import catboost
from catboost import CatBoostClassifier
from catboost.utils import get_roc_curve
# 获取类别特征索引（假设X_train是DataFrame）
cat_features_idx = [i for i, col in enumerate(X_train.columns) 
                    if X_train[col].dtype == 'object']  
# 创建并训练一个CatBoost分类器
model = CatBoostClassifier(iterations=100, 
                           depth=6, 
                           learning_rate=0.1, 
                           loss_function='Logloss', 
                           cat_features=cat_features_idx)
model.fit(X_train, y_train)
# 假设 df 是你的 Pandas DataFrame
train_data = catboost.Pool(data=X_train, label=y_train)
# 获取特征重要性评估
feature_importance = model.get_feature_importance(data=train_data, type='LossFunctionChange')

# 打印特征重要性得分
for i, score in enumerate(feature_importance):
    print(f'Feature {i}: {score}')

0:	learn: 0.6438165	total: 156ms	remaining: 15.4s
1:	learn: 0.6041469	total: 508ms	remaining: 24.9s
2:	learn: 0.5726872	total: 585ms	remaining: 18.9s
3:	learn: 0.5483656	total: 677ms	remaining: 16.2s
4:	learn: 0.5308079	total: 757ms	remaining: 14.4s
5:	learn: 0.5150613	total: 838ms	remaining: 13.1s
6:	learn: 0.5034853	total: 916ms	remaining: 12.2s
7:	learn: 0.4940764	total: 1.01s	remaining: 11.6s
8:	learn: 0.4863646	total: 1.1s	remaining: 11.1s
9:	learn: 0.4802039	total: 1.22s	remaining: 11s
10:	learn: 0.4752099	total: 1.33s	remaining: 10.8s
11:	learn: 0.4711910	total: 1.42s	remaining: 10.4s
12:	learn: 0.4678764	total: 1.62s	remaining: 10.8s
13:	learn: 0.4654842	total: 2.01s	remaining: 12.4s
14:	learn: 0.4631880	total: 2.13s	remaining: 12.1s
15:	learn: 0.4612211	total: 2.22s	remaining: 11.7s
16:	learn: 0.4594985	total: 2.44s	remaining: 11.9s
17:	learn: 0.4580763	total: 2.56s	remaining: 11.7s
18:	learn: 0.4568508	total: 2.66s	remaining: 11.3s
19:	learn: 0.4559494	total: 2.74s	remaining:

In [8]:
# 3. 模型建立与训练
from catboost import CatBoostClassifier
model = CatBoostClassifier(iterations=100, cat_features=feature_list, eval_metric='AUC', logging_level='Verbose', learning_rate=0.05, depth=6, l2_leaf_reg=5, loss_function='CrossEntropy')
model.fit(X_train.loc[:, feature_list], y_train, 
          eval_set=(X_validation.loc[:, feature_list], y_validation), plot=False)

0:	test: 0.6942840	best: 0.6942840 (0)	total: 742ms	remaining: 1m 13s
1:	test: 0.6977473	best: 0.6977473 (1)	total: 1.29s	remaining: 1m 3s
2:	test: 0.7025593	best: 0.7025593 (2)	total: 1.78s	remaining: 57.7s
3:	test: 0.7036131	best: 0.7036131 (3)	total: 2.28s	remaining: 54.7s
4:	test: 0.7046930	best: 0.7046930 (4)	total: 2.68s	remaining: 51s
5:	test: 0.7057725	best: 0.7057725 (5)	total: 3.07s	remaining: 48.1s
6:	test: 0.7061735	best: 0.7061735 (6)	total: 3.43s	remaining: 45.6s
7:	test: 0.7063476	best: 0.7063476 (7)	total: 3.79s	remaining: 43.6s
8:	test: 0.7069699	best: 0.7069699 (8)	total: 4.33s	remaining: 43.8s
9:	test: 0.7072312	best: 0.7072312 (9)	total: 4.78s	remaining: 43s
10:	test: 0.7068720	best: 0.7072312 (9)	total: 5.26s	remaining: 42.6s
11:	test: 0.7072285	best: 0.7072312 (9)	total: 5.64s	remaining: 41.4s
12:	test: 0.7076547	best: 0.7076547 (12)	total: 6.01s	remaining: 40.2s
13:	test: 0.7081026	best: 0.7081026 (13)	total: 6.42s	remaining: 39.4s
14:	test: 0.7080742	best: 0.708

In [ ]:
# 4.测试集预测
preds = model.predict_proba(test_data[feature_list])
preds


array([[0.92087715, 0.07912285],
       [0.71542053, 0.28457947],
       [0.62919857, 0.37080143],
       ...,
       [0.74120586, 0.25879414],
       [0.68797578, 0.31202422],
       [0.9513777 , 0.0486223 ]])

In [10]:
test_data['isDefault'] = preds[:, 1]  # 取正类的概率存储
test_data[['id', 'isDefault']].to_csv('submission.csv', index=False)